* Colab jupyter notebook끼리 import하기 위한 선언

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Colab Notebooks/밑바닥부터 시작하는 딥러닝 1

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/밑바닥부터 시작하는 딥러닝 1


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import sys, os

In [3]:
sys.path.append('/content/drive/MyDrive/Colab Notebooks/밑바닥부터 시작하는 딥러닝 1/sample')

In [4]:
sys.path

['',
 '/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/drive/MyDrive/Colab Notebooks/밑바닥부터 시작하는 딥러닝 1/sample']

### 합성곱 신경망 CNN
* 합성곱 계층 / 풀링 계층으로 나뉨
* Conv - ReLu - Pooling
* 마지막엔 ReLU - Affien - Softmax
* 합성곱 계층은 공간적 정보를 갖는 방면 / 완전연결 계층은 데이터의 형상이 무시됨(가로 * 세로)
* https://jerife.github.io/2021-05-10-cnn/
* 위 링크에 정리돼 있음

In [5]:
x = np.random.rand(10, 1, 28, 28)
x.shape

(10, 1, 28, 28)

In [6]:
print(x[0].shape)
print(x[1].shape) 

(1, 28, 28)
(1, 28, 28)


In [14]:
from sample.common.util import im2col

In [16]:
x1 = np.random.rand(1, 3, 7, 7)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape)

(9, 75)


In [18]:
x1 = np.random.rand(10, 3, 7, 7)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape)

(90, 75)


In [19]:
class Convolution:
  def __init__(self, W, b, stride=1, pad=0):
    self.W = W
    self.b = b
    self.stride = stride
    self.pad = pad
  
  def forward(self, x):
    FN, C, FH, FW = self.W.shape
    N, C, H, W = x.shape

    out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
    out_h = int(1 + (W + 2*self.pad - FW) / self.stride)

    col = im2col(x, FH, FW, self.stride, self.pad)
    col_W = self.W.reshape(FN, -1).T # 필터 전개
    out = np.dot(col, col_W) + self.b

    out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2) # 인덱스로 축의 순서 변경

    return out

In [ ]:
class Pooling:
  def __init__(self, pool_h, pool_W, stride=1, pad=0):
    self.pool_h = pool_h
    self.pool_W = pool_W
    self.stride = stride
    self.pad = pad

  def forward(self, x):
    N, C, H, W = x.shape
    out_h = int(1 + (H - self.pool_h) / self.stride)
    out_w = int(1 + (W - self.pool_w) / self.stride)

    # 전개 1
    col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
    col = col.reshape(-1, self.pool_h*self.pool_w)

    # 최댓값
    out = np.max(col, axis=1)

    # 성형
    out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)

    return out